In [1]:
from imapclient import IMAPClient
import email

from dotenv import load_dotenv
import os


In [2]:
# Load environment variables from .env file
load_dotenv()

EMAIL_ADDRESS = os.getenv("EMAIL_ADDRESS")
EMAIL_APP_PASSWORD = os.getenv("EMAIL_APP_PASSWORD")
EMAIL_HOST = os.getenv("EMAIL_HOST")

In [3]:
server = IMAPClient(EMAIL_HOST, use_uid=True)
_ = server.login(EMAIL_ADDRESS, EMAIL_APP_PASSWORD)

In [15]:
server.select_folder("INBOX", readonly=True)
selected_mails = server.search(["UNSEEN", ["FROM", "noreply@redditmail.com"]])

In [35]:
from bs4 import BeautifulSoup

for mail in selected_mails:
    # print(mail)
    response = server.fetch(mail, "RFC822")
    email_data = response[mail][b'RFC822']
    email_message = email.message_from_bytes(email_data)
    # print(email_message)
    subject = email_message.get("Subject")
    from_ = email_message.get("From")
    print(f"From: {from_}\nSubject: {subject}\n")
    # Initialize a variable for the body text
    body = None
    
    # If the email has multiple parts, iterate over each part
    if email_message.is_multipart():
        for part in email_message.walk():
            # Look for text/plain or text/html content
            if part.get_content_type() == "text/plain":
                body = part.get_payload(decode=True).decode(part.get_content_charset(), errors="replace")
                break
            elif part.get_content_type() == "text/html":
                html = part.get_payload(decode=True).decode(part.get_content_charset(), errors="replace")
                # Use BeautifulSoup to extract the text content from the HTML
                soup = BeautifulSoup(html, "html.parser")
                body = soup.get_text()
                break
    else:
        # If it's a single-part email
        if email_message.get_content_type() == "text/plain":
            body = email_message.get_payload(decode=True).decode(email_message.get_content_charset(), errors="replace")
        elif email_message.get_content_type() == "text/html":
            html = email_message.get_payload(decode=True).decode(email_message.get_content_charset(), errors="replace")
            # Use BeautifulSoup to extract the text content from the HTML
            soup = BeautifulSoup(html, "html.parser")
            body = soup.get_text()
    
    # Print the extracted text content
    if body:
        print(f"Body:\n{"\n".join([line for line in body.splitlines() if line.strip()])}\n")
    else:
        print("No body content found.\n")

From: Reddit <noreply@redditmail.com>
Subject: Updates to Reddit's Privacy Policy and User Agreement

Body:
Email Template
Equivalent_Vehicle83
We’re writing to let you know that we recently updated Reddit’s
User Agreement and
Privacy Policy.
While we encourage you to review the updated documents in full, here are some of the highlights:
We reorganized sections of our Privacy Policy for improved clarity.
We included language in the Privacy Policy to reflect our participation in the EU-US Data Privacy Framework.
We clarified what content you submit is non-public.
We updated the Privacy Policy to better reflect our current products, like Reddit Pro.
For users in the EEA, United Kingdom, and Switzerland, we updated the User Agreement to reflect that the Reddit products and services are provided by Reddit Netherlands B.V.
Updates to our Privacy Policy take effect immediately.
The updated User Agreement takes effect September 16, 2024 and will apply to you if you use Reddit after that date.